In [6]:
import io
from base64 import encodebytes
from PIL import Image
from flask import jsonify

def encoder_image(image_path):
    pil_img = Image.open(image_path, mode='r') # reads the PIL image
    byte_arr = io.BytesIO()
    pil_img.save(byte_arr, format='PNG') # convert the PIL image to byte array
    encoded_img = encodebytes(byte_arr.getvalue()).decode('ascii') # encode as base64
    return encoded_img

def show_images():
    img_streams = []
    for file in os.listdir(cache_dir):
        file_path = os.path.join(cache_dir, file)
        img_stream = encoder_image(file_path)
        img_streams.append(img_stream)
    
    return jsonify({'result': img_streams})

data_json = show_images()


NameError: name 'Image' is not defined

In [ ]:
from ps_model import load_model
from test_personsearch_prw_sys import get_prw_dataset_info, get_prw_data, search_performance_prw
from test_personsearch_cuhk_sys import get_cuhk_dataset_info, get_cuhk_data, search_performance_cuhk
import torch
from __init__ import *
info_sota = get_info_sota()

In [ ]:
# PRW_Dataset, pname_to_attribute, gt_roidb, name_to_det_feat_prw = get_prw_dataset_info(info_sota['PRW'])
query_data_loader, psdb_dataset, name_to_det_feat_cuhk_sota = get_cuhk_dataset_info(info_sota['CUHK'])

In [ ]:
idx = 10
# data = get_prw_data(PRW_Dataset, pname_to_attribute, idx)
data = get_cuhk_data(query_data_loader, idx)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

proposals = data['proposals'][0]._data[0][0]
scale_factor = data['img_metas'][0]._data[0][0]['scale_factor']
# proposals = proposals / scale_factor
proposals = proposals.tolist()[0]

filename = data['img_metas'][0]._data[0][0]['filename']
img = Image.open(filename)
plt.figure(figsize=(8, 8))
plt.xticks([])
plt.yticks([])
plt.imshow(img)

ax = plt.gca()
down_left_x, down_left_y = proposals[0], proposals[1]
width = proposals[2] - proposals[0]
height = proposals[3] - proposals[1]
rect = plt.Rectangle((down_left_x, down_left_y), width, height, fill=False, edgecolor='yellow', linewidth=1)
ax.add_patch(rect)
save_root_dir = "/home/linhuadong/DNFNet/tools/system/cache"
if not os.path.exists(save_root_dir):
    os.makedirs(save_root_dir)
# plt.savefig(os.path.join(save_root_dir, 'query' + '.png'), bbox_inches='tight')


In [ ]:
model = load_model(info_sota['PRW'], 2)

In [ ]:
idx = 2
data = get_prw_data(PRW_Dataset, pname_to_attribute, idx)
with torch.no_grad():
    result = model(return_loss=False, rescale=True, **data)
entry = search_performance_prw(result, data, pname_to_attribute, name_to_det_feat_prw, gt_roidb)

In [ ]:
def visualize(image_root, probe_img, probe_roi, color, save_name):
    file_root = os.path.join(image_root, probe_img)
    img = Image.open(file_root)
    plt.figure(figsize=(8, 8))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    
    ax = plt.gca()
    down_left_x, down_left_y = probe_roi[0], probe_roi[1]
    width = probe_roi[2] - probe_roi[0]
    height = probe_roi[3] - probe_roi[1]
    rect = plt.Rectangle((down_left_x, down_left_y), width, height, fill=False, edgecolor=color, linewidth=1)
    ax.add_patch(rect)
    save_root_dir = "/home/linhuadong/DNFNet/tools/system/cache"
    if not os.path.exists(save_root_dir):
        os.makedirs(save_root_dir)
    plt.savefig(os.path.join(save_root_dir, save_name))

def search(results):
    colors = ['red', 'green', 'yellow']
    search_result = []
    visualize(results['image_root'], results['probe_img'], results['probe_roi'], colors[2]) # query
    for j in range(5):
        info = results['gallery'][j]
        img = visualize(results['image_root'], info['img'], info['roi'][:4], colors[info['correct']], save_name=f'top-{j+1}')  # gallery
    return True

search(entry)
